-- This is the better code for it : )
by: Benjamin Karic 30.01.2024

In [ ]:
import folium
import pandas as pd
import geopandas as gpd
from folium.plugins import MarkerCluster

# Read the first Excel file (with blue markers)
blue_markers_file_path = r"C:\Users\bkari\Downloads\Output_2.xlsx"
blue_markers_df = pd.read_excel(blue_markers_file_path)

# Read the second Excel file (with pink markers)
pink_markers_file_path = r"C:\Users\bkari\Downloads\MTI_converted.xlsx"
pink_markers_df = pd.read_excel(pink_markers_file_path)

# Drop rows where 'Lat_Y' or 'Long_X' is null for both datasets
blue_markers_df = blue_markers_df.dropna(subset=['Lat_Y', 'Long_X'])
pink_markers_df = pink_markers_df.dropna(subset=['Lat_Y', 'Long_X'])

# Create GeoDataFrames from the DataFrames
blue_markers_gdf = gpd.GeoDataFrame(blue_markers_df, geometry=gpd.points_from_xy(blue_markers_df['Long_X'], blue_markers_df['Lat_Y']))
pink_markers_gdf = gpd.GeoDataFrame(pink_markers_df, geometry=gpd.points_from_xy(pink_markers_df['Long_X'], pink_markers_df['Lat_Y']))

# Define latitude and longitude ranges that approximately cover Austria
austria_lat_range = (46.372, 49.021)
austria_lon_range = (9.530, 17.160)

# Clip the blue markers to stay within the boundaries of Austria
blue_markers_gdf = blue_markers_gdf.cx[austria_lon_range[0]:austria_lon_range[1], austria_lat_range[0]:austria_lat_range[1]]

# Calculate the map center based on the clipped blue markers GeoDataFrame
map_center = [blue_markers_gdf['Lat_Y'].mean(), blue_markers_gdf['Long_X'].mean()]

# Create a folium map centered around Austria
my_map = folium.Map(location=map_center, zoom_start=8, tiles='OpenStreetMap')

# Fit the map to the boundaries of Austria
my_map.fit_bounds([[austria_lat_range[0], austria_lon_range[0]],
                   [austria_lat_range[1], austria_lon_range[1]]])

# Add blue markers to the map with clustering!
blue_cluster = MarkerCluster().add_to(my_map)
for idx, row in blue_markers_gdf.iterrows():
    popup_html = f"<b>Sitecode:</b> {row['Sitecode']}<br>"
    for col in blue_markers_df.columns:
        if col.startswith('Points'):
            popup_html += f"<b>{col}:</b> {row[col]}<br>"
    folium.Marker(location=[row['Lat_Y'], row['Long_X']], popup=popup_html, icon=folium.Icon(color='blue')).add_to(blue_cluster)

# Add pink markers to the map with clustering
pink_cluster = MarkerCluster().add_to(my_map)
for idx, row in pink_markers_gdf.iterrows():
    popup_html = f"<b>UNIQUE_IDENTIFIER:</b> {row['UNIQUE_IDENTIFIER']}<br><b>STANDORTTYP:</b> {row['STANDORTTYP']}<br><b>ARGE_MTI:</b> {row['ARGE_MTI']}<br>"
    folium.Marker(location=[row['Lat_Y'], row['Long_X']], popup=popup_html, icon=folium.Icon(color='pink')).add_to(pink_cluster)

# Save the map as an HTML file
html_file_path = r'C:\Users\bkari\Downloads\combined_map_updated.html'
my_map.save(html_file_path)
print(f"Map saved to {html_file_path}")
